# Exercises on Bayesian regularized regression

**Authors**: original R Tutorial created by Sara van Erp and Duco Veen; Python translation by Florian Metwaly.

This is the answer file to the 'exercise_Bayesian_regularization.ipybn' notebook. You can find the correct answers under the corresponding exercise number.

### Exercise 1

In [ ]:
model = bmb.Model("Rings ~ 0 + Sex + Length + Diameter + Height + Whole_weight + Shucked_weight + Viscera_weight + Shell_weight", data=train)
fit_default = model.fit()

### Exercise 2

In [ ]:
az.summary(fit_default)

### Exercise 3

In [ ]:
ridge_prior = bmb.Prior("Normal", mu=0, sigma=0.1)

model = bmb.Model(
    "Rings ~ Sex + Length + Diameter + Height + Whole_weight + Shucked_weight + Viscera_weight + Shell_weight",
    data=train,
    priors={
        "Sex": ridge_prior,
        "Length": ridge_prior,
        "Diameter": ridge_prior,
        "Height": ridge_prior,
        "Whole_weight": ridge_prior,
        "Shucked_weight": ridge_prior,
        "Viscera_weight": ridge_prior,
        "Shell_weight": ridge_prior
    },
    center_predictors=False
)

# Fit the model
fit_ridge = model.fit(random_seed=SEED)

# Summary
az.summary(fit_ridge)

### Exercise 4

In [ ]:
vars_to_plot = ["Diameter", "Whole_weight", "Shell_weight"]

# Default priors
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
az.plot_posterior(fit_default, var_names=vars_to_plot,
                  hdi_prob=0.95, ax=axes)
for ax in axes:
    ax.set_xlim(-50, 75)
fig.suptitle("Default Priors")
plt.tight_layout()
plt.show()

# Ridge priors
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
az.plot_posterior(fit_ridge, var_names=vars_to_plot,
                  hdi_prob=0.95, ax=axes)
for ax in axes:
    ax.set_xlim(-50, 75)
fig.suptitle("Ridge Priors")
plt.tight_layout()
plt.show()

### Exercise 5

In [ ]:
# Default priors
fig, ax = plt.subplots(figsize=(6, 4))
az.plot_posterior(fit_default, var_names=["sigma"],
                  hdi_prob=0.95, ax=ax)
ax.set_xlim(1, 4.5)
fig.suptitle("Default Priors")
plt.tight_layout()
plt.show()

# Ridge priors
fig, ax = plt.subplots(figsize=(6, 4))
az.plot_posterior(fit_ridge, var_names=["sigma"],
                  hdi_prob=0.95, ax=ax)
ax.set_xlim(1, 4.5)
fig.suptitle("Ridge Priors")
plt.tight_layout()
plt.show()

### Exercise 6

In [ ]:
variables = ["Sex", "Length", "Diameter", "Height",
             "Whole_weight", "Shucked_weight", "Viscera_weight", "Shell_weight"]

for var in variables:
    fig, axes = plt.subplots(1, 3, figsize=(15, 4), sharey=True)

    # Ridge
    az.plot_posterior(fit_ridge, var_names=[var], ax=axes[0])
    axes[0].set_title("Ridge Prior")
    axes[0].set_xlim(-5, 5)
    axes[0].set_ylim(0, 5)

    # Student-t
    az.plot_posterior(fit_t, var_names=[var], ax=axes[1])
    axes[1].set_title("Student-t Prior")
    axes[1].set_xlim(-5, 5)
    axes[1].set_ylim(0, 5)

    # Lasso
    az.plot_posterior(fit_lasso, var_names=[var], ax=axes[2])
    axes[2].set_title("Laplace (Lasso) Prior")
    axes[2].set_xlim(-5, 5)
    axes[2].set_ylim(0, 5)

    fig.suptitle(f"Posterior of '{var}' across Priors", y=1.05, fontsize=16)
    plt.tight_layout()
    plt.show()


#### Bonus

In [ ]:
def extract_posterior_samples(idata, model_name, vars):
    samples = az.extract(idata, var_names=vars)
    df = samples.to_dataframe()
    df["model"] = model_name
    return df

df_ridge = extract_posterior_samples(fit_ridge, "Ridge", variables)
df_t = extract_posterior_samples(fit_t, "Student-t", variables)
df_lasso = extract_posterior_samples(fit_lasso, "Lasso", variables)

# Combine all data
df_all = pd.concat([df_ridge, df_t, df_lasso])

# Melt into long-form for seaborn
df_melt = df_all.melt(id_vars="model", value_vars=variables, var_name="parameter", value_name="value")

# Plot
g = sns.FacetGrid(df_melt, col="parameter", sharex=False, sharey=False,
                  height=4, aspect=1.2, col_wrap=4)  # optional col_wrap for layout
g.map_dataframe(sns.kdeplot, x="value", hue="model", fill=True, common_norm=False, alpha=0.5)
g.add_legend(title="Model")
g._legend.set_bbox_to_anchor((1, 1))  # Optional: move it to the right
g.set_titles("{col_name}")
g.set_axis_labels("Posterior value", "Density")
plt.tight_layout()
plt.show()

### Exercise 7

In [ ]:
model_simple = bmb.Model("Rings ~ Length", data=train,
                         center_predictors=False)
fit_simple = model_simple.fit(random_seed=SEED)

az.summary(fit_simple)

### Exercise 8

In [ ]:
model_normal_prior = bmb.Model("Rings ~ Length", data=train,
                         priors = {"Length": bmb.Prior("Normal", mu=0, sigma=10)},
                         center_predictors=False)
fit_normal_prior = model_normal_prior.fit(random_seed=SEED)

az.summary(fit_normal_prior)
# There is practically no influence of this prior; it is relatively uninformative given the scale of the variable.

### Exercise 9

In [ ]:
## Frequentist Regression
X_2 = train_new["Length_cm"]
y_2 = train_new["Rings"]
X_2 = sm.add_constant(X_2)

fit2 = sm.OLS(y_2, X_2).fit()

print(fit2.summary())

## Bayesian Regression
model_normal_prior2 = bmb.Model("Rings ~ Length", data=train_new,
                         priors = {"Length_cm": bmb.Prior("Normal", mu=0, sigma=10)},
                         center_predictors=False)
fit_normal_prior2 = model_normal_prior2.fit(random_seed=SEED)

az.summary(fit_normal_prior2)
# Now, the effect is shrunken heavily!

## Exercise 10

In [ ]:
ridge_prior_scaled = bmb.Prior("Normal", mu=0, sigma=0.1)

model_ridge_scaled = bmb.Model(
    "Rings ~ Sex + Length + Diameter + Height + Whole_weight + Shucked_weight + Viscera_weight + Shell_weight",
    data=train,
    priors={
        "Sex": ridge_prior,
        "Length": ridge_prior,
        "Diameter": ridge_prior,
        "Height": ridge_prior,
        "Whole_weight": ridge_prior,
        "Shucked_weight": ridge_prior,
        "Viscera_weight": ridge_prior,
        "Shell_weight": ridge_prior
    },
    center_predictors=True
)

# Fit the model
fit_ridge_scaled = model_ridge_scaled.fit(random_seed=SEED)

# Summary
az.summary(fit_ridge_scaled)

## Exercise 10

In [ ]:
ridge_prior = bmb.Prior("Normal", mu=0, sigma=0.1)

model_ridge_scaled = bmb.Model(
    "Rings ~ Sex + Length + Diameter + Height + Whole_weight + Shucked_weight + Viscera_weight + Shell_weight",
    data=train,
    priors={
        "Sex": ridge_prior,
        "Length": ridge_prior,
        "Diameter": ridge_prior,
        "Height": ridge_prior,
        "Whole_weight": ridge_prior,
        "Shucked_weight": ridge_prior,
        "Viscera_weight": ridge_prior,
        "Shell_weight": ridge_prior
    },
    center_predictors=True
)

# Fit the model
fit_ridge_scaled = model_ridge_scaled.fit(random_seed=SEED)

# Summary
az.summary(fit_ridge_scaled, hdi_prob=0.95)

# Now, we would select Shell_weight and Shucked_weight based on the 95% CI!

## Plot them
# Not centered
az.plot_posterior(fit_ridge, var_names=["Shell_weight", "Shucked_weight"],
                  hdi_prob=0.95)
plt.suptitle("Not centered")
plt.tight_layout()
plt.show()

# Centered
az.plot_posterior(fit_ridge_scaled, var_names=["Shell_weight", "Shucked_weight"],
                  hdi_prob=0.95)
plt.suptitle("Centered")
plt.tight_layout()
plt.show()